## A stateless oracle (5): setting up an exchange that uses the oracle
#### 09.5 Winter School on Smart Contracts
##### Peter Gruber (peter.gruber@usi.ch)
2022-02-15
* Part 5: Using the Oracle (I): Setting up an exchange
* Parts 1-4 are only relevant if you want to **create** an Oracle
* This example shows how Bob can start a business (an exchange) that uses the oracle

## Setup
See notebook 04.1, the lines below will always automatically load functions in `algo_util.py`, the five accounts and the Purestake credentials

In [100]:
# Loading shared code and credentials
import sys, os

codepath = '..'+os.path.sep+'..'+os.path.sep+'sharedCode'
sys.path.append(codepath)
from algo_util import *
cred = load_credentials()

# Shortcuts to directly access the 3 main accounts
Alice   = cred['Alice']
Bob     = cred['Bob']
Charlie = cred['Charlie']
Dina    = cred['Dina']

In [120]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk.future import transaction
from algosdk.future.transaction import Multisig
from algosdk.future.transaction import PaymentTxn, MultisigTransaction
from algosdk.future.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn
from algosdk.future.transaction import LogicSig, LogicSigTransaction, LogicSigAccount

import algosdk.error
import json
import base64
import hashlib

In [102]:
from pyteal import *

In [103]:
# Initialize the algod client (Testnet or Mainnet)
algod_client = algod.AlgodClient(algod_token='', algod_address=cred['algod_test'], headers=cred['purestake_token'])

In [104]:
print(Alice['public'])
print(Bob['public'])
print(Charlie['public'])

VK6CCXY4IFHIJAVMRVS543LJQEQKOJO6YQ4DZNV3D2XJI4ETYBN5354EQU
CPUT3Z5CI3XOIZ4ARSGUFQD7V4YGYJW5BFAZMXX5YOV4KJCKI6MBCDY5XM
BY5K2AYO7R3G66ICY6SJ2JFVLRMIX677EAEEKDBTJZGP6Q4JVNZRDXDBKA


In [105]:
import json
import requests
import pandas as pd
import numpy as np
import time

In [106]:
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()

## Set up specific addresses

Two options
1. Use your own oracle --> update the addresses and the oracle_id
2. Use the course oracle --> keep the addresses and the oracle_id

In [1]:
USDC_id   = 10458941           # USDC on Algorand Testnet

# Update the following, if you want to use your own oracle
oracle_id = 77534697 

# from WSC 09.4
price_sig = {'program': b'AyAD6av8JOgHBCYBIAbye8Ew9QvLSn3Bp51f1ZBM7rIIanCZErjePwlGFTDFMQAoEjEUKBIQMREiEhAxASMOEDEQJBIxCTIDEhAxIDIDEhAQQw==', 
             'sig': 'l87eZLMhhdkpPHos+B/eZJC/f01NoHBy2jW4B3ofYMalN9e/lyszBGdLt7h0XH0tECqTegG1wJasZGGQa2LCCg==', 
             'public': b'A3ZHXQJQ6UF4WST5YGTZ2X6VSBGO5MQINJYJSEVY3Y7QSRQVGDCQ====', 
             'public_addr': 'A3ZHXQJQ6UF4WST5YGTZ2X6VSBGO5MQINJYJSEVY3Y7QSRQVGDCSQSRQOE'}

reserve_sig = {'program': b'AyAD6av8JOgHBCYBIEqOKk1XaR/CRfnNvSFE+b0Gj7cb4Lnb1L9qynBTYu9fMQAoEjEUKBIQMREiEhAxASMOEDEQJBIxCTIDEhAxIDIDEhAQQw==', 
               'sig': 'KezTvw0s0Y9JTePlFfCgeKWVdcH6OAuzX4Bg/D1cjW0ikrbAcG3q+kSbpuOKTetK9tZNXJufxHG8DCxXXJ7+CQ==', 
               'public': b'JKHCUTKXNEP4ERPZZW6SCRHZXUDI7NY34C45XVF7NLFHAU3C55PQ====', 
               'public_addr': 'JKHCUTKXNEP4ERPZZW6SCRHZXUDI7NY34C45XVF7NLFHAU3C55P7Q6YPZU'}

## Get information about the oracle coin

In [108]:
print('https://testnet.algoexplorer.io/asset/{}'.format(oracle_id))

https://testnet.algoexplorer.io/asset/77534697


## How to use the Oracle
* Bob creates an exchange that uses the Oracle
* The exchange is an Atomic Swap, where the exchange rate obtains from the oracle
* Bob creates a smart signature that uses transaction groups

## The structure of Bobs's smart signature
* `Txn[0]` a transaction of the *Price* account with itself to obtain the price
* `Txn[1]` a transaction of the *Reserve* account to verify the price
    * Criterion: the amounts of `Txn[0]` and `Txn[1]` must add up exactly to the total supply of Oracle Coins
* `Txn[2]` the ALGO transaction
* `Txn[3]` the USDC transaction
    * Criterion: the amounts of `Txn[2]` and `Txn[3]` must correctly reflect the exchange rate
    * The exchange rate is obtained from `Txn[0]` 

**Note** transactions 0 and 1 are the oracle part, transactions 2 and 3 are the actual atomic swap



#### Step 1: Create Smart Signature

In [354]:
### Based on Atomic Swap of 06.4_WSC

oracle_condition = And (
    Gtxn[0].type_enum() == TxnType.AssetTransfer,         # Oracle = ASA transfer ...
    Gtxn[1].type_enum() == TxnType.AssetTransfer,
    Gtxn[0].xfer_asset() == Int(oracle_id),               # ... of oracle coin
    Gtxn[1].xfer_asset() == Int(oracle_id),           
    Gtxn[0].sender() == Addr(price_sig['public_addr']),      # Ensure correct order 
    Gtxn[1].sender() == Addr(reserve_sig['public_addr']),
    # Verification condition: must move ALL coins ... total = 1000 coins in small units
    Gtxn[0].asset_amount() + Gtxn[1].asset_amount() == Int(int( 1e3 * 1e6 ))   
    )    

exchange_condition = And (
    Gtxn[2].type_enum() == TxnType.Payment,            # Txn2 is in ALGOs
    Gtxn[2].xfer_asset() == Int(0),
    Gtxn[3].type_enum() == TxnType.AssetTransfer,      # Txn3 is in USDC
    Gtxn[3].xfer_asset() == Int(USDC_id),
    # Exchange rate in small units (note: Algo, Oracle and USDC *all* have 6 decimals)
    # Exchange rate is taken from Gtxn[0].asset_amount()
    # ALGO_amount * USD_per_ALGO == USD_amount
    Gtxn[2].amount() * Gtxn[0].asset_amount() / Int(int(1e6)) == Gtxn[3].asset_amount(),
    Gtxn[2].amount() >= Int(int(1e5)),                 # Min tx size to make it impossible to profit from rounding errors
    Gtxn[3].asset_amount() >= Int(int(1e5)),           # Min tx size to make it impossible to profit from rounding errors
)

safety_condition = And(
    # safety conditions ONLY for veding, opt-in must handle its own safety!
    Global.group_size() == Int(4),                     # 0 and 1 for oracle, 2 and 3 for atomic swap
    Gtxn[2].rekey_to() == Global.zero_address(),
    Gtxn[2].close_remainder_to() == Global.zero_address(),
    Gtxn[3].rekey_to() == Global.zero_address(),
    Gtxn[3].close_remainder_to() == Global.zero_address(), 
)
    
optin_condition = And(
        Global.group_size() == Int(1),                # Opt-in is single transaction
        Txn.type_enum() ==TxnType.AssetTransfer,      # Opt-in is ASA transfer
        Txn.asset_amount() == Int(0),                 # Payout impossible, opt-in is OK
        Txn.xfer_asset() == Int(USDC_id),             # Only opt into USDc
        Txn.rekey_to() == Global.zero_address(),
        Txn.close_remainder_to() == Global.zero_address(),
    )

import random
a = Int( random.randrange(2**32-1) )
random_cond = ( a == a )

fee_cond = Txn.fee() <= Int(1000)

payout_cond = Or(
    And(Txn.asset_receiver() == Addr(Bob['public']),          # Payout to Bob in USDC 
        Txn.type_enum() ==TxnType.AssetTransfer,
        Txn.xfer_asset() == Int(USDC_id)),
    And(Txn.receiver() == Addr(Bob['public']),                # ... or ALGO
        Txn.type_enum() == TxnType.Payment)
)
    
exchange_pyteal = And(
    random_cond, 
    fee_cond, 
    If(
        Global.group_size() == Int(1),             # condition
        Or(optin_condition,                        # then-expression
           payout_cond),
        And(exchange_condition,                    # else-expression
            oracle_condition,
            safety_condition
           )
        )
    )

In [355]:
exchange_teal = compileTeal(exchange_pyteal, Mode.Signature, version=3)
#print(Exchange_teal)

In [356]:
Exchange = algod_client.compile(exchange_teal)
Exchange

{'hash': 'TXUUALJEISZ6YJFW7SNCBWFAQVJLVRSVAP6I4NP5CW4NME63HTEKDHM3AY',
 'result': 'AyAK3ajsR+gHAQAEva7+BMCEPaCNBumr/CSAlOvcAyYDIAbye8Ew9QvLSn3Bp51f1ZBM7rIIanCZErjePwlGFTDFIEqOKk1XaR/CRfnNvSFE+b0Gj7cb4Lnb1L9qynBTYu9fIBPpPeeiRu7kZ4CMjULAf68wbCbdCUGWXv3Dq8UkSkeYIiISMQEjDhAyBCQSQACOMwIQJBIzAhElEhAzAxAhBBIQMwMRIQUSEDMCCDMAEgshBgozAxISEDMCCCEHDxAzAxIhBw8QMwAQIQQSMwEQIQQSEDMAESEIEhAzAREhCBIQMwAAKBIQMwEAKRIQMwASMwESCCEJEhAQMgQhBBIzAiAyAxIQMwIJMgMSEDMDIDIDEhAzAwkyAxIQEEIAPDIEJBIxECEEEhAxEiUSEDERIQUSEDEgMgMSEDEJMgMSEDEUKhIxECEEEhAxESEFEhAxByoSMRAkEhARERBD'}

#### Step 2: Exchange must opt-in and must be funded

##### Step 2a: Funding in ALGO

In [357]:
# Prepare, sign, send, wait
sp = algod_client.suggested_params()

# Minimum is 0.201 ALGO for 2 x min_holdings + TX_fee for opt_in
# However, we want to do an exchange and therefore have to fund more
amt = int(2 * 1e6)
txn = transaction.PaymentTxn(sender=Bob['public'], sp=sp, receiver=Exchange['hash'], amt=amt)
stxn = txn.sign(Bob['private'])
txid = algod_client.send_transaction(stxn)
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  20490357.
Waiting for round 20490357 to finish.
Waiting for round 20490358 to finish.
Transaction IBWRKLIGPJWAENYEWW6HNGGZFXRHEVUJB3AMN7DYZ5TZ4OZISDHQ confirmed in round 20490359.


##### Step 2b: Opt-In USDC

In [358]:
# Prepare, sign, send, wait
sp = algod_client.suggested_params()
txn = AssetTransferTxn(Exchange['hash'], sp, Exchange['hash'], 0, USDC_id)
encodedProg = Exchange['result'].encode()
program = base64.decodebytes(encodedProg)
lsig = LogicSig(program)
stxn = LogicSigTransaction(txn, lsig)
txid = algod_client.send_transaction(stxn)
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  20490359.
Waiting for round 20490359 to finish.
Waiting for round 20490360 to finish.
Transaction X7F2JEDAHGKHCY2U5EMTPVZ7EW7CXPHGMVLX5VJSGCAZIKFT724Q confirmed in round 20490361.


#### Step 2c: Funding in USDC
* Get USDC here: https://dispenser.testnet.aws.algodev.network
* Bob can get USDC here: https://usdcfaucet.com (select "ALGO" and run five times)

In [359]:
# Prepare, sign, send, wait
amt = int(2*1e6)
sp = algod_client.suggested_params()
txn = AssetTransferTxn(
    sender=Bob['public'],
    sp=sp,
    receiver=Exchange['hash'],               
    amt=amt,
    index=USDC_id)
stxn = txn.sign(Bob['private'])
txid = algod_client.send_transaction(stxn)
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  20490361.
Waiting for round 20490361 to finish.
Waiting for round 20490362 to finish.
Transaction ZXJHKT7LMC4EGY6Y4K7KDSUCSOX27SUCRB7DGVT7ZNMYABDWFX6A confirmed in round 20490363.


## Test the Smart Sig

In [360]:
# Customer Information
Customer = Charlie
amt_ALGO = -int(1.5 * 1e6)    # positive --> buyALGOs
                             # negative --> sellALGOs

In [361]:
asset_holdings(algod_client, Charlie['public'])

[{'amount': 7.183,
  'unit': 'ALGO',
  'asset-id': 0,
  'name': 'Algorand',
  'decimals': 6},
 {'amount': 304.845024,
  'unit': 'USDC',
  'asset-id': 10458941,
  'name': 'USDC',
  'decimals': 6}]

In [362]:
# get current holdings
holdings_Price = asset_holdings(algod_client, price_sig['public_addr'])
price_oracle = [holding['amount'] for holding in holdings_Price if holding['unit']=='USDALGO'][0]
price_oracle = int(1e6*price_oracle)
holdings_Reserve = asset_holdings(algod_client, reserve_sig['public_addr'])
reserve_oracle = [holding['amount'] for holding in holdings_Reserve if holding['unit']=='USDALGO'][0]
reserve_oracle = int(1e6*reserve_oracle)

print(price_oracle)
print(reserve_oracle)

805969
999194031


#### Transaction 0: Price sends to itself

In [363]:
my_sig = price_sig
amt = price_oracle       # "everything"

# Step 0: Recreate Smart Signature --> see WSC 06.6
lsigPrice = LogicSigAccount(base64.decodebytes(my_sig['program']))
lsigPrice.lsig.sig = my_sig['sig']
lsigPrice.sigkey = base64.b32decode(my_sig['public'])

# Step 1: prepare Txn
sp = algod_client.suggested_params()          
txn_0 = AssetTransferTxn(sender = my_sig['public_addr'], sp=sp, receiver=my_sig['public_addr'], amt=amt, index=oracle_id)

#### Transaction 1: Reserve sends to itself

In [364]:
my_sig = reserve_sig
amt = reserve_oracle       # "everything"

# Step 0: Recreate Smart Signature --> see WSC 06.6
lsigReserve = LogicSigAccount(base64.decodebytes(my_sig['program']))
lsigReserve.lsig.sig = my_sig['sig']
lsigReserve.sigkey = base64.b32decode(my_sig['public'])

# Step 1: prepare Txn
sp = algod_client.suggested_params()          
txn_1 = AssetTransferTxn(sender = my_sig['public_addr'], sp=sp, receiver=my_sig['public_addr'], amt=amt, index=oracle_id)

#### Transaction 2: Algo transaction

In [365]:
sp = algod_client.suggested_params()          

if amt_ALGO > 0:                
    # Customer buys ALGO from Exchange
    txn_2 = PaymentTxn(sender = Exchange['hash'], sp=sp, receiver=Customer['public'], amt=abs(amt_ALGO))
else:
    txn_2 = PaymentTxn(sender = Customer['public'], sp=sp, receiver=Exchange['hash'], amt=abs(amt_ALGO))


#### Transaction 3: USDC transaction

In [366]:
sp = algod_client.suggested_params()
amt_USDC = int(amt_ALGO * price_oracle / 1e6)
print(amt_USDC)
print("condition")
print(int(amt_ALGO  * price_oracle / 1e6))
print(int(amt_USDC))

-1208953
condition
-1208953
-1208953


In [367]:
if amt_ALGO > 0:                
    # Customer sends USDC to Exchange
    txn_3 = AssetTransferTxn(sender = Customer['public'], sp=sp, receiver=Exchange['hash'], amt=abs(amt_USDC), index=USDC_id)
else:
    txn_3 = AssetTransferTxn(sender = Exchange['hash'], sp=sp, receiver=Customer['public'], amt=abs(amt_USDC), index=USDC_id)

#### Create Transaction group
See WSC 06.4, step 5.1c

In [368]:
gid = transaction.calculate_group_id([txn_0, txn_1, txn_2, txn_3])
txn_0.group = gid
txn_1.group = gid
txn_2.group = gid
txn_3.group = gid

#### Sign Individual transactions
See WSC 06.4, step 5.2

In [369]:
# txn_0 and txn_1 are signed by their smart signatures
# encodedProg = Vending['result'].encode()              
# program = base64.decodebytes(encodedProg)
# lsig = LogicSig(program)
# stxn_2 = LogicSigTransaction(txn_2, lsig)

stxn_0 = transaction.LogicSigTransaction(txn_0, lsigPrice)
stxn_1 = transaction.LogicSigTransaction(txn_1, lsigReserve)

encodedProg = Exchange['result'].encode()              
program = base64.decodebytes(encodedProg)
Exchange_lsig = LogicSig(program)
#stxn_2 = LogicSigTransaction(txn_2, lsig)

if amt_ALGO > 0:                
    # Customer buys ALGO from Exchange
    stxn_3 = txn_3.sign(Customer['private'])
    stxn_2 = transaction.LogicSigTransaction(txn_2, Exchange_lsig)
else:
    stxn_2 = txn_2.sign(Customer['private'])
    stxn_3 = transaction.LogicSigTransaction(txn_3, Exchange_lsig)

In [370]:
# Step 5.3: assemble transaction group and send
signed_group =  [stxn_0, stxn_1, stxn_2, stxn_3]
txid = algod_client.send_transactions(signed_group)

In [371]:
# Step 5d: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  20490365.
Waiting for round 20490365 to finish.
Transaction LHZ3TU7OVYTDEYYDN5SAPIAX7LIGAIUTUO332UZHDCCBB4UJMADQ confirmed in round 20490366.
